In [1]:
import numpy as np
import pandas as pd
from metadata import data_type
from src.pipe_store import (
    data_loader,
    date_parser,
    clean_string_strip,
    set_data_types,
    integer_encoder,
    sort_values_per_client,
    datetime2int,
    one_hot_encoder,
)

%load_ext autoreload
%autoreload 2

In [2]:
data_path = '/Users/Danial/Downloads/assesment_file2_churn.csv'
df = (
    data_loader(data_path, parse_dates=['MONTH_PERIOD'], date_parser=date_parser)
    .pipe(set_data_types, data_type)
    .pipe(clean_string_strip, 'AGE_CLASS', 'HOMEBANK_COLOUR', 'LOYALITY')
    .pipe(sort_values_per_client, 'MONTH_PERIOD')
)


# df.MONTH_PERIOD = df.MONTH_PERIOD.apply(lambda x: (x - pd.to_datetime('2011-1-1').to_period('M')).n)

Step: data_loader | Shape: (100056, 28) | Computation Time: 0.62821s
Step: set_data_types | Shape: (100056, 28) | Computation Time: 0.159731s
Step: clean_string_strip | Shape: (100056, 28) | Computation Time: 0.005075s
Step: sort_values_per_client | Shape: (100056, 28) | Computation Time: 22.736606s


# Missing Values
Variables with missing values are: ACCOUNTMODEL, AGE_CLASS, HOMEBANK_COLOUR, LOYALITY

Missing Completely at Random (MCAR)-> random sampling from variable distribution

Missing at Random (MAR)-> random sampling from variable distribution or predict missing class via Logistic Regression

Missing not at Random (MNAR): Should not be imputed

### Adopted Strategy

Consider all the missingess as MNAR and 

In [ ]:
col_with_missing = ['ACCOUNTMODEL', 'AGE_CLASS', 'HOMEBANK_COLOUR', 'LOYALITY']
for col in col_with_missing:
    print(f'Column:{col}, missingness: {df[col].isna().sum() * 100 / len(df): 0.3} %')

In [ ]:
# def impute_age_class(df):
#     df_imp = df.copy(deep=True)
#     df_imp.AGE_CLASS = df_imp.AGE_CLASS.replace('Leeftijd_onbekend', np.NAN)
#     ids_age_nan = df_imp[df_imp.AGE_CLASS.isna()].CUSTOMER_ID.unique()
#     for id in ids_age_nan:
#         ind = df_imp.CUSTOMER_ID.eq(id)
#         filled = df_imp[ind].AGE_CLASS.fillna(method='backfill')
#         if not filled.isna().sum():
#             filled = df_imp[ind].AGE_CLASS.fillna(method='ffill')
#         df_imp.loc[ind ,'AGE_CLASS'] = filled
#     return df_imp
# df_imp = impute_age_class(df)

In [ ]:
# (df.AGE_CLASS.value_counts(normalize=True) * 100).plot.bar()

In [ ]:
# (df.ACCOUNTMODEL.value_counts(normalize=True) * 100).plot.bar();

# EDA 

In [ ]:
from src.feature_selection import variable_variances, variable_variances_per_client
variable_variances(df, include_label=False)
# 'Record_Count', 'TARGET' should be discarded from the data set

In [ ]:
var_client = variable_variances_per_client(df)
# var_client.var()

In [ ]:
var_client

In [ ]:
var_client.MORTGAGE_IND.value_counts(normalize=True).plot.bar()

In [ ]:
# churned_clients = var_client[var_client['CHURNED_IND'] != 0].index.tolist()
# com_churned_clients = var_client[var_client['COMMERCIALLY_CHURNED'] != 0].index.tolist()
# both_churned_label = var_client[(var_client['CHURNED_IND'] != 0) & (var_client['COMMERCIALLY_CHURNED'] != 0)].index.tolist()
# either_churned_label = var_client[(var_client['CHURNED_IND'] != 0) | (var_client['COMMERCIALLY_CHURNED'] != 0)].index.tolist()
# single_churned_label = var_client[(var_client['CHURNED_IND'] != 0) & (var_client['COMMERCIALLY_CHURNED'] == 0)].index.tolist()
# single_churned_label_com = var_client[(var_client['CHURNED_IND'] == 0) & (var_client['COMMERCIALLY_CHURNED'] != 0)].index.tolist()
# not_churned_clients = var_client[(var_client['CHURNED_IND'] == 0) & (var_client['COMMERCIALLY_CHURNED'] == 0)].index.tolist()

# Feature Selection

From previous section we know to drop: 'Record_Count', 'TARGET'

### Univariate Correlation

In [ ]:
from src.feature_selection import plot_corr_cat
from sklearn.preprocessing import LabelEncoder
df_corr = df_imp.copy(deep=True) 
encoding_cols = ['CLIENTGROUP', 'ACCOUNTMODEL', 'AGE_CLASS', 'HOMEBANK_COLOUR', 'LOYALITY']
for col in encoding_cols:
    encoder = LabelEncoder()
    df_corr[col] = encoder.fit_transform(df_corr[col])

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif
col_drop = ['MONTH_PERIOD', 'Record_Count', 'TARGET', 'CHURNED_IND', 'COMMERCIALLY_CHURNED', 'CUSTOMER_ID', 'CROSS_SELL_SCORE']
X, y1, y2 = df_corr.drop(col_drop, axis=1), df['CHURNED_IND'], df['COMMERCIALLY_CHURNED']
k_best = SelectKBest(chi2, k=10).fit(X, y1)
selected_cols = k_best.get_feature_names_out()
selected_cols

In [ ]:
# import matplotlib.pyplot as plt
# from seaborn import  heatmap
# corr_mat = df_corr.corr()
# plt.figure(figsize=(12, 8))
# heatmap(corr_mat, cmap='RdYlGn', annot=True)

# Feature Engineering

# Modelling

## Time-to-Event Approach

We need to ensure that time column "MONTH_PERIOD" is engineered to number of months after the start of observation/data collection
df.MONTH_PERIOD = df.MONTH_PERIOD.apply(lambda x: (x.to_period('M') - pd.to_datetime('2011-1-1').to_period('M')).n)

In [ ]:
# df_surv = df.copy(deep=True)
# df_surv = df_surv.drop(['TARGET', 'Record_Count'], axis=1)

# dfs = {}
# for id in df_surv.CUSTOMER_ID.unique():
#     df_id = df_surv[df_surv.CUSTOMER_ID.eq(id)].sort_values(by='MONTH_PERIOD', ascending=False)
#     df_id = df_id.assign(
#         start=df['MONTH_PERIOD'],
#         stop=df['MONTH_PERIOD'] + 1,
#         event=df['CHURNED_IND'].astype(bool),
#         id=df['CUSTOMER_ID'],
#     )
#     slice_size = df_id.CHURNED_IND.lt(1).sum()
#     # samples =  slice_size + 1 if slice_size < 24 
#     dfs[id] = df_id.tail(slice_size + 1)
# df_surv_final = (
#     pd.concat([df.reset_index(drop=True) for id, df in dfs.items()])
#     .drop(['CUSTOMER_ID', 'MONTH_PERIOD', 'CHURNED_IND', 'COMMERCIALLY_CHURNED'], axis=1)
# )

# Stacking

In [ ]:
from collections import defaultdict
churn_col = 'CHURNED_IND'
horizon = 6 # Time horizon (Even number) to find dominant recent past states per column
df_s = df.drop(['TARGET', 'Record_Count'], axis=1)
bool_cols = [
    'PAYMENT_IND', 'SAVING_IND', 'INVESTMENTS_IND', 'LENDING_IND', 'INSURANCE_LIFE_IND',
    'INSURANCE_NONLIFE_IND', 'MORTGAGE_IND', 'PACKAGE_IND',]

cat_cols = [
    'CREDIT_CLASS', 'DEBIT_CLASS', 'INVESTED_CAPITAL_CLASS', 'SAVINGS_CAPITAL_CLASS', 
    'MIN_FEED_CLASS', 'REVENUES_CLASS', 'PAYMENT_ACTIVITIES_CODE',  'CLIENTGROUP', 'ACCOUNTMODEL',
    'AGE_CLASS', 'HOMEBANK_COLOUR', 'LOYALITY',]

num_cols = ['CROSS_SELL_SCORE',]

_dics = defaultdict(lambda: defaultdict(dict))
ids_skipped = []
try:
    for id in df_s.CUSTOMER_ID.unique():
        df_id = (
            df_s[df_s.CUSTOMER_ID.eq(id)]
            .sort_values(by='MONTH_PERIOD', ascending=False)
            .reset_index(drop=True)
        )
        # client switches active -> churn -> active 
        if df_id[churn_col].diff().abs().sum() > 1:
            print(f'Client {id}, active -> churn -> active ')
            ids_skipped.append(id)
            continue
        # client starts with churn status 
        if df_id[churn_col].values[-1]:
            print(f'Client {id} churn -> active')
            ids_skipped.append(id)
            continue

        if 1 in df_id[churn_col].values:
            _dics[id]['event'] = 1
            ind_lt_1 = df_id[churn_col].lt(1)
            ind_churn = df_id[df_id[churn_col].lt(1)].index.min()
            _dics[id]['churn_time'] = df_id.loc[ind_churn - 1, 'MONTH_PERIOD']
            for col in set(df_id.columns) - { 'MONTH_PERIOD', 'CHURNED_IND', 'COMMERCIALLY_CHURNED', 'CUSTOMER_ID'}:
                vals = df_id[ind_lt_1][col].values
                _dics[id][col] = vals[0] # Most recent value
                _dics[id][col + '_CHANGED'] = 1 if len(set(vals[1:horizon])) > 1 else 0
                # _dics[id][col + '_PAST_STATE'] = lst[0] if len(lst[1:]) == 0 else max(vals[1:horizon], key=lst.count) # Most frequent value past horizon 
        else:
            _dics[id]['event'] = 0
            _dics[id]['churn_time'] = pd.to_datetime('2013-1-1')
            for col in set(df_id.columns) - { 'MONTH_PERIOD', 'CHURNED_IND', 'COMMERCIALLY_CHURNED', 'CUSTOMER_ID'}:
                vals = df_id[col].values
                _dics[id][col] = vals[0] # Most recent value
                _dics[id][col + '_CHANGED'] = 1 if len(set(vals[1:horizon])) > 1 else 0
                # _dics[id][col + '_PAST_STATE'] = lst[0] if len(lst[1:]) == 0 else max(vals[1:horizon], key=lst.count) # Most frequent value past horizon 
        
except:
    print(id)

df_red = pd.DataFrame(_dics).T.rename_axis('id').reset_index()

In [ ]:
df_red

### New dataset
45 churn samples experienced either rechurn or rejoin. For the sake of avoiding bias these samples are removed frome the dataset.

In [ ]:
df_red = (
    df_red
    .pipe(datetime2int, 'churn_time')
    .reset_index(drop=True)
    .fillna('unknown')
)
len(ids_skipped)

# Seasonality [pattern in churn time]

In [ ]:
df_red[df_red.event == 1]['churn_time'].value_counts().sort_index().plot.bar()


In [ ]:
# df_red.groupby('event')['PACKAGE_IND'].value_counts()
df_red.columns

In [ ]:
df_red['churn_time'] = df_red['churn_time'].astype('float16')
df_red['event'] = df_red['event'].astype('float16')

In [ ]:
from lifelines import KaplanMeierFitter, NelsonAalenFitter, WeibullFitter
from lifelines.plotting import add_at_risk_counts
import matplotlib.pyplot as plt
%matplotlib inline

def plot_churn_risk(df, col, estimator, ax=None, T:str='churn_time', E:str='event',  horizon=23, at_risk=False):
    """ Plot univariate churn risk """

    if col not in df.columns:
        raise KeyError(f'{col} not in data frame')
    if not ax:
        fig, ax = plt.subplots(figsize=(6, 5))
    timeline = np.linspace(0, horizon, 1000)
    vals = set(df[col].values)
    estimators = [estimator(label=col.split('_')[0]+'_'+str(val)) for val in vals]
    inds = [df_red[col] == val if str(val) != 'nan' else df_red[col].isna() for val in vals ]
    estimators = [estimator.fit(df_red[inds[i]][T], df_red[inds[i]][E], timeline=timeline) 
                    for i, estimator in enumerate(estimators)]
    # if type(estimator) in ['NelsonAalenFitter', 'WeibullFitter']:
    for estimator in estimators:
        estimator.plot(ci_show=False, ax=ax)
    if at_risk:
        add_at_risk_counts(*estimators, ax=ax, fontsize=10, rows_to_show=['At risk'])
    ax.set_ylim(0.5)
    ax.set_xlim([0, 23])
    ax.set_xlabel('Months', fontsize=10)
    ax.set_ylabel('Probability of Retention', fontsize=10)

In [ ]:
bool_cols

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 4))
# ('INSURANCE_LIFE_IND', 'INSURANCE_LIFE_IND'), ('MORTGAGE_IND', 'PACKAGE_IND'), ('INVESTMENTS_IND', 'LENDING_IND')
for i, j in [('PAYMENT_IND', 'SAVING_IND')]: 
    plot_churn_risk(df_red, i, ax=ax[0], estimator=KaplanMeierFitter, at_risk=True)
    plot_churn_risk(df_red, j, ax=ax[1], estimator=KaplanMeierFitter, at_risk=True)

In [ ]:
# 'CREDIT_CLASS', 'DEBIT_CLASS', 'INVESTED_CAPITAL_CLASS', 'SAVINGS_CAPITAL_CLASS', 
# 'MIN_FEED_CLASS', 'REVENUES_CLASS', 'PAYMENT_ACTIVITIES_CODE', 'CROSS_SELL_SCORE', 'CLIENTGROUP', 'ACCOUNTMODEL',
# 'AGE_CLASS', 'HOMEBANK_COLOUR', 'LOYALITY']
fig, ax = plt.subplots(1,2, figsize=(14, 4))
plot_churn_risk(df_red, 'HOMEBANK_COLOUR', ax=ax[0], estimator=KaplanMeierFitter, at_risk=True)
plot_churn_risk(df_red, 'LOYALITY', ax=ax[1], estimator=KaplanMeierFitter, at_risk=True)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14, 4))
# 'CREDIT_CLASS', 'DEBIT_CLASS', 'INVESTED_CAPITAL_CLASS', 'SAVINGS_CAPITAL_CLASS', 
# 'MIN_FEED_CLASS', 'REVENUES_CLASS', 'PAYMENT_ACTIVITIES_CODE', 'CROSS_SELL_SCORE', 'CLIENTGROUP', 'ACCOUNTMODEL',
# 'AGE_CLASS', 'HOMEBANK_COLOUR', 'LOYALITY']
plot_churn_risk(df_red, 'AGE_CLASS', ax=ax[0], estimator=KaplanMeierFitter, at_risk=True)
plot_churn_risk(df_red, 'AGE_CLASS_CHANGED', ax=ax[1], estimator=KaplanMeierFitter, at_risk=True)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14, 4))
plot_churn_risk(df_red, 'CREDIT_CLASS', ax=ax[0], estimator=KaplanMeierFitter, at_risk=True)
plot_churn_risk(df_red, 'DEBIT_CLASS', ax=ax[1], estimator=KaplanMeierFitter, at_risk=True)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14, 4)) 
plot_churn_risk(df_red, 'INVESTED_CAPITAL_CLASS', ax=ax[0], estimator=KaplanMeierFitter, at_risk=True)
plot_churn_risk(df_red, 'SAVINGS_CAPITAL_CLASS', ax=ax[1], estimator=KaplanMeierFitter, at_risk=True)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14, 4)) 
plot_churn_risk(df_red, 'MIN_FEED_CLASS', ax=ax[0], estimator=KaplanMeierFitter, at_risk=True)
plot_churn_risk(df_red, 'REVENUES_CLASS', ax=ax[1], estimator=KaplanMeierFitter, at_risk=True)

In [ ]:
# 'CLIENTGROUP', 'ACCOUNTMODEL',
fig, ax = plt.subplots(1,2, figsize=(14, 4)) 
plot_churn_risk(df_red, 'PAYMENT_ACTIVITIES_CODE', ax=ax[0], estimator=KaplanMeierFitter, at_risk=True)
plot_churn_risk(df_red, 'CROSS_SELL_SCORE', ax=ax[1], estimator=KaplanMeierFitter, at_risk=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5)) 
plot_churn_risk(df_red, 'CLIENTGROUP', ax=ax, estimator=KaplanMeierFitter, at_risk=True)

In [ ]:
# 0101: 13%, 0307:23 %,  0105: 42%


In [ ]:
df_red.AGE_CLASS.unique()

# Cox Partial Hazard Model

In [ ]:
cat_cols = [
    'CREDIT_CLASS', 'DEBIT_CLASS', 'INVESTED_CAPITAL_CLASS', 'SAVINGS_CAPITAL_CLASS', 
    'MIN_FEED_CLASS', 'REVENUES_CLASS', 'PAYMENT_ACTIVITIES_CODE', 
    # 'CROSS_SELL_SCORE', 
    'CLIENTGROUP', 'ACCOUNTMODEL', 'AGE_CLASS', 'HOMEBANK_COLOUR', 'LOYALITY']

df_cox = (
    df_red
    .pipe(one_hot_encoder, *cat_cols)
    [[
        'event', 'churn_time', 
        'PAYMENT_IND',
        'SAVING_IND',
        # 'CREDIT_CLASS_0', 
        'CREDIT_CLASS_1', #
        'CREDIT_CLASS_2',
        # 'DEBIT_CLASS_0', #
        # 'DEBIT_CLASS_1', #
        # 'DEBIT_CLASS_2', #
        'SAVINGS_CAPITAL_CLASS_0', #
        'SAVINGS_CAPITAL_CLASS_2',
        'INVESTED_CAPITAL_CLASS_0',
        'MIN_FEED_CLASS_0', # 
        'MIN_FEED_CLASS_1',
        'REVENUES_CLASS_3', #
        'PAYMENT_ACTIVITIES_CODE_0', 
        # 'CLIENTGROUP_0105', #
        'CLIENTGROUP_0307',
        'CLIENTGROUP_0101', #
        'CROSS_SELL_SCORE',  # 'CROSS_SELL_SCORE_0',
        'AGE_CLASS_Leeftijd_onbekend', 
        'HOMEBANK_COLOUR_unknown', 
        'HOMEBANK_COLOUR_Rood',
        'LOYALITY_unknown', 
        'LOYALITY_Rood', 
    ]]
)

In [ ]:
# from seaborn import heatmap 
# plt.figure(figsize=(10, 7))
# heatmap(df_cox.corr(), cmap='RdYlGn')

In [ ]:
df_cox.columns

In [ ]:
from lifelines import CoxPHFitter
cph = CoxPHFitter()
cph.fit(df_cox, duration_col='churn_time', event_col='event')
cph.print_summary() 

In [ ]:
df_cox

In [ ]:
df_cox

# Classifier

In [ ]:
from sklearn.model_selection import cross_validate, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from src.pipe_store import sklearn_adapter
random_state = 42

X, y = sklearn_adapter(df_cox, label='event')
# regs = [ LogisticRegression(), RandomForestClassifier()]
X.drop('churn_time', axis=1, inplace=True)
regs =  [ LogisticRegression(), RandomForestClassifier(), GaussianNB()]

num_cols = ['CROSS_SELL_SCORE']
column_transformer_scaler = ColumnTransformer([
    ('Scaler', StandardScaler(), num_cols), 
], remainder='passthrough')

results = {}
for reg in regs:

    pipeline = Pipeline([
        ('scaler', column_transformer_scaler),
        ('Model', reg),
    ], verbose=False)

    kfs = KFold(n_splits=5, shuffle=True)
    # For the list of all metrics visit: https://scikit-learn.org/stable/modules/model_evaluation.html
    metrics = ['recall', 'precision', 'roc_auc', 'accuracy', 'f1'] 
    scores = cross_validate(pipeline, X, y, cv=kfs, scoring=metrics)
    # We will not use cross_val_score as it can only accept one metric
    # print(scores)
    reg_name = type(reg).__name__
    results[reg_name] = {key: round(np.mean(val), 3) for key, val in scores.items()}
pd.DataFrame(results).T

In [ ]:
from sklearn.model_selection import train_test_split
X, y = sklearn_adapter(df_cox, label='event')
X.drop('churn_time', axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)
clf = GaussianNB() # LogisticRegression() #GaussianNB()
pipeline = Pipeline([
    ('scaler', column_transformer_scaler),
    ('Model', clf),
], verbose=False)

pipeline.fit(X_train, y_train)

# Model Evaluation

In [ ]:
from src.model_evaluation import (
    plot_roc_curve,
    plot_confusion_matrix,
    plot_precision_recall_curve,
    print_scores,
)

from sklearn.model_selection import train_test_split
X, y = sklearn_adapter(df_cox, label='event')
X.drop('churn_time', axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)
clf = GaussianNB() # LogisticRegression() #GaussianNB()
pipeline = Pipeline([
    ('scaler', column_transformer_scaler),
    ('Model', clf),
], verbose=False)

pipeline.fit(X_train, y_train)

plot_confusion_matrix(pipeline, X_train, y_train)
fig, ax = plt.subplots(1,2, figsize=(11, 4))
plot_precision_recall_curve(pipeline, X_train, y_train, ax[0])
plot_roc_curve(pipeline, X_train, y_train, ax[1])
print_scores(pipeline, X_test, y_test)

In [ ]:
plot_confusion_matrix(pipeline, X_test, y_test)
print_scores(pipeline, X_test, y_test)
fig, ax = plt.subplots(1,2, figsize=(10, 4))
plot_roc_curve(pipeline, X_test, y_test, ax[0])
plot_precision_recall_curve(pipeline, X_test, y_test, ax[1])


In [ ]:
# from src.model_evaluation import plot_calibration
# plot_calibration(pipeline, X_test, y_test, n_bins=5, strategy='uniform')

In [ ]:

# forest_importances = pd.Series(rcf.feature_importances_, index=X.columns)

# fig, ax = plt.subplots()
# forest_importances.plot.bar( ax=ax)
# ax.set_title("Feature importances using MDI")
# ax.set_ylabel("Mean decrease in impurity")
# fig.tight_layout()

In [ ]:
X